
# Laboratório: DW de Saúde Pública — COVID‑19 (Brasil)

**Tema:** Saúde Pública (COVID‑19)  
**Bases:**  
1) **OWID COVID** (casos/óbitos por país) — CSV público  
2) **OWID Vaccinations** (vacinação por país) — CSV público  
3) **BrasilAPI Feriados** (enriquecimento de `dim_date` com `is_holiday`)

> Objetivo: consolidar dados diários de COVID e vacinação para o Brasil em um **DW (Star Schema)**, realizar **ETL** com pandas e aplicar **clustering** para segmentar períodos epidemiológicos.



## Bases (links)
- OWID COVID: `https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv`  
- OWID Vaccinations: `https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv`  
- BrasilAPI (feriados por ano): `https://brasilapi.com.br/api/feriados/v1/{ANO}`

> Se a internet estiver bloqueada, baixe os CSVs e coloque ao lado do notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

%matplotlib inline

OUT_DIR = Path("/content/drive/MyDrive/dw_health_output_aula")
OUT_DIR.mkdir(exist_ok=True)

# Nova seção
## Modelo conceitual (Star Schema)

Fato **`fact_covid_daily`** (granularidade: **dia-país**):  
**Chaves:** `date_sk`, `location_sk`  
**Medidas:** `new_cases`, `new_deaths`, `new_vaccinations`, `people_fully_vaccinated`, `stringency_index` (quando disponível)

Dimensões:  
- **`dim_date`**: `date_sk`, `date`, `year`, `month`, `day`, `is_holiday`  
- **`dim_location`**: `location_sk`, `iso_code`, `location`, `continent`, `population`, `population_density`


<pre style="font-size:13px; white-space:pre; overflow-x:auto;">
                   +-------------------+
                   |     dim_date      |
                   +-------------------+
                   | date_sk (PK)      |
                   | date              |
                   | year              |
                   | month             |
                   | day               |
                   | is_holiday        |
                   +-------------------+
                          |
                          | (FK)
                          |
+-------------------+     |     +-------------------+
|  dim_location     |     |     |  fact_covid_daily |
+-------------------+     |     +-------------------+
| location_sk (PK)  |-----+-----| date_sk (FK)      |
| iso_code          |           | location_sk (FK)  |
| location          |           | new_cases         |
| continent         |           | new_deaths        |
| population        |           | new_vaccinations  |
| population_density|           | people_fully_vacc |
+-------------------+           | stringency_index  |
                                +-------------------+
</pre>

In [3]:
def read_csv_web_or_local(url):
    fname = Path(url.split("/")[-1])
    full_output = OUT_DIR / fname

    if full_output.exists():
        print("✔ Lido do cache:", full_output)
        return pd.read_csv(full_output)

    else:
        print("✔ Lido da web:", url)
        try:
            df = pd.read_csv(url)
            print("✔ Lido da web:", url)
            if not full_output.exists():
                df.to_csv(full_output, index=False)
            return df
        except Exception as e:
            print("Web falhou:", e)

OWID_COVID = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
OWID_VAX = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"

covid = read_csv_web_or_local(OWID_COVID)
vax = read_csv_web_or_local(OWID_VAX)

covid.shape, vax.shape

✔ Lido do cache: /content/drive/MyDrive/dw_health_output_aula/owid-covid-data.csv
✔ Lido do cache: /content/drive/MyDrive/dw_health_output_aula/vaccinations.csv


((429435, 67), (196246, 16))

In [4]:
covid.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [5]:
vax.columns

Index(['location', 'iso_code', 'date', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'daily_vaccinations_per_million', 'daily_people_vaccinated',
       'daily_people_vaccinated_per_hundred'],
      dtype='object')

In [6]:
covid.dtypes

,0
iso_code,object
continent,object
location,object
date,object
total_cases,float64
...,...
population,int64
excess_mortality_cumulative_absolute,float64
excess_mortality_cumulative,float64
excess_mortality,float64


In [7]:
covid["date"] = pd.to_datetime(covid["date"])
vax["date"] = pd.to_datetime(vax["date"])

In [8]:
print(covid.iloc[1000].to_json(indent=2))

{
  "iso_code":"AFG",
  "continent":"Asia",
  "location":"Afghanistan",
  "date":1664582400000,
  "total_cases":198283.0,
  "new_cases":0.0,
  "new_cases_smoothed":171.57,
  "total_deaths":7796.0,
  "new_deaths":0.0,
  "new_deaths_smoothed":0.29,
  "total_cases_per_million":4886.36,
  "new_cases_per_million":0.0,
  "new_cases_smoothed_per_million":4.23,
  "total_deaths_per_million":192.12,
  "new_deaths_per_million":0.0,
  "new_deaths_smoothed_per_million":0.01,
  "reproduction_rate":0.91,
  "icu_patients":null,
  "icu_patients_per_million":null,
  "hosp_patients":null,
  "hosp_patients_per_million":null,
  "weekly_icu_admissions":null,
  "weekly_icu_admissions_per_million":null,
  "weekly_hosp_admissions":null,
  "weekly_hosp_admissions_per_million":null,
  "total_tests":null,
  "new_tests":null,
  "total_tests_per_thousand":null,
  "new_tests_per_thousand":null,
  "new_tests_smoothed":null,
  "new_tests_smoothed_per_thousand":null,
  "positive_rate":null,
  "tests_per_case":null,
  "